In [1]:
#pip install librosa soundfile numpy sklearn pyaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
#conda install librosa soundfile numpy sklearn pyaudio #python package for music and audio analysis


Note: you may need to restart the kernel to use updated packages.



CondaValueError: invalid package specification: #python



In [35]:
# pip install pydub
# from pydub import AudioSegment

SyntaxError: invalid syntax (<ipython-input-35-554805d18422>, line 1)

In [75]:
# pip install hmmlearn

In [42]:
#conda install -c anaconda pyaudio

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\R.N.Mandal\anaconda3

  added / updated specs:
    - pyaudio


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.12.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2020.12.5-py38ha~ --> anaconda::certifi-2020.6.20-py38_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['neutral', 'happy', 'sad', 'angry']

In [4]:
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles
dirName = './Ravdess'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

1440

In [100]:
# import speech_recognition as sr
# r=sr.Recognizer()
# for file in range(0 , len(listOfFiles) , 1):
#     with sr.AudioFile(listOfFiles[file]) as source:
#         audio = r.listen(source)
#         try:
#             text = r.recognize_google(audio)
#             print(text)
#         except:
#             print('error')

In [5]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\R.N.Mandal\\Desktop\\AVER\\clean_ravdess\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9) # split the data to training and testing and return it

In [102]:
# def envelope(y , rate, threshold):
#     mask=[]
#     y=pd.Series(y).apply(np.abs)
#     y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
#     for mean in y_mean:
#         if mean>threshold:
#             mask.append(True)
#         else:
#             mask.append(False)
#     return mask

In [6]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.10)

In [7]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))
#print(np.shape(x_train),np.shape(x_test), np.shape(y_train),np.shape(y_test))

(604, 68)


In [8]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [9]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [10]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [11]:
#Predict for the test set
y_pred=model.predict(x_test)

In [12]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.47%


In [22]:
# import glob,pickle
# from tqdm import tqdm
# from scipy.io import wavfile
# for file in tqdm(glob.glob(r'C:\Users\R.N.Mandal\Desktop\AVER\Ravdess\\**\\*.wav')):
#     file_name = os.path.basename(file)
#     signal , rate = librosa.load(file, sr=16000)
#     mask = envelope(signal,rate, 0.0005)
#     wavfile.write(filename= r'C:\Users\R.N.Mandal\Desktop\AVER\clean_ravdess\\'+str(file_name), rate=rate,data=signal[mask])

100%|██████████| 1440/1440 [00:30<00:00, 47.37it/s]


In [13]:
Pkl_Filename = "NewModel.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [3]:
# Load the Model back from file
with open("NewModel.pkl", 'rb') as file:  
    Emotion_Model = pickle.load(file)

Emotion_Model

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [15]:
y_pred=Emotion_Model.predict(x_test)
y_pred

array(['happy', 'angry', 'angry', 'sad', 'happy', 'angry', 'sad', 'happy',
       'angry', 'neutral', 'neutral', 'happy', 'happy', 'happy', 'sad',
       'happy', 'happy', 'sad', 'happy', 'angry', 'sad', 'neutral',
       'happy', 'happy', 'neutral', 'sad', 'angry', 'happy', 'neutral',
       'sad', 'angry', 'sad', 'angry', 'neutral', 'angry', 'angry',
       'neutral', 'angry', 'neutral', 'angry', 'neutral', 'happy', 'sad',
       'angry', 'happy', 'happy', 'angry', 'sad', 'angry', 'angry',
       'angry', 'happy', 'sad', 'happy', 'sad', 'sad', 'angry', 'sad',
       'angry', 'neutral', 'neutral', 'neutral', 'neutral', 'sad',
       'angry', 'happy', 'happy', 'happy'], dtype='<U7')

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.80      0.89      0.84        18
       happy       0.75      0.75      0.75        20
     neutral       0.85      0.85      0.85        13
         sad       0.67      0.59      0.62        17

    accuracy                           0.76        68
   macro avg       0.77      0.77      0.77        68
weighted avg       0.76      0.76      0.76        68



In [17]:
import wave
obj = wave.open('03-01-02-01-01-02-06.wav','r')
print( "Number of channels",obj.getnchannels())
print ( "Sample width",obj.getsampwidth())
print ( "Frame rate.",obj.getframerate())
print ("Number of frames",obj.getnframes())
print ( "parameters:",obj.getparams())
obj.close()

Number of channels 1
Sample width 2
Frame rate. 16000
Number of frames 65666
parameters: _wave_params(nchannels=1, sampwidth=2, framerate=16000, nframes=65666, comptype='NONE', compname='not compressed')


In [18]:
import wave
obj = wave.open('chunkchannel_1.wav','r')
print( "Number of channels",obj.getnchannels())
print ( "Sample width",obj.getsampwidth())
print ( "Frame rate.",obj.getframerate())
print ("Number of frames",obj.getnframes())
print ( "parameters:",obj.getparams())
obj.close()

Number of channels 1
Sample width 2
Frame rate. 44100
Number of frames 2520138
parameters: _wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=2520138, comptype='NONE', compname='not compressed')


In [19]:
from scipy.io import wavfile

fs, data = wavfile.read('chunk4.wav')            # reading the file

wavfile.write('chunkchannel_1.wav', fs, data[:, 0])   # saving first column which corresponds to channel 1
wavfile.write('chunkchannel_2.wav', fs, data[:, 1])

In [20]:
file = 'chunkchannel_1.wav' #channel set to 1
ans =[]
new_feature= extract_feature(file, mfcc=True, chroma=True, mel=True)
print (new_feature)
print(np.shape(new_feature))
ans.append(new_feature)

ans = np.array(ans)
print(np.shape(ans))
try:
  print(Emotion_Voice_Detection_Model.predict([ans]))
except:
  print("An exception occurred")
#Emotion_Voice_Detection_Model.predict([ans])

[-2.63951111e+02  1.09196487e+02 -2.35478687e+01  1.53101168e+01
 -1.56890392e+01 -3.79026628e+00 -2.22364407e+01 -8.35200214e+00
 -2.25083160e+01 -2.18681183e+01 -1.61357307e+01 -1.97900791e+01
 -9.17027760e+00 -1.14603491e+01 -8.87360191e+00  1.06001461e+00
 -1.38177061e+00 -8.73414898e+00 -1.24653852e+00 -6.16787815e+00
 -4.20024014e+00 -7.21634007e+00 -5.15412951e+00 -5.07060528e+00
 -5.98534203e+00 -2.96882010e+00 -4.33370209e+00 -2.41858289e-01
 -1.64866611e-01  1.48652041e+00  6.32396698e-01  4.22976112e+00
  8.50229681e-01  2.04047418e+00  1.69705141e+00  4.24935758e-01
  1.78301191e+00  1.63527334e+00  3.66667223e+00  4.63462532e-01
  6.04041874e-01  6.12379193e-01  5.74256361e-01  5.11513352e-01
  5.01899481e-01  4.90761399e-01  4.83331531e-01  5.03887892e-01
  5.31942785e-01  5.63771427e-01  5.65476298e-01  5.77016950e-01
  3.34906834e-03  1.21145304e-02  3.79848890e-02  1.24816351e-01
  5.26822627e-01  2.34029484e+00  8.51986027e+00  1.67659035e+01
  2.46528625e+01  1.84085

In [21]:
file = 'chunkchannel_1.wav' #channel set to 1
new_feature= extract_feature(file, mfcc=True, chroma=True, mel=True)
try:
  print(Emotion_Voice_Detection_Model.predict([new_feature]))
except:
  print("An exception occurred")

An exception occurred


In [22]:
from scipy.io import wavfile

fs, data = wavfile.read('C:/Users/R.N.Mandal/Desktop/AVER/audio_chunks/audio_NICKVUJICIC/chunk2.wav')            # reading the file

wavfile.write('chunk2_1.wav', fs, data[:, 0])   # saving first column which corresponds to channel 1
wavfile.write('chunk2_2.wav', fs, data[:, 1])

In [23]:
file = 'chunk2_1.wav' #channel set to 1
new_feature= extract_feature(file, mfcc=True, chroma=True, mel=True)
try:
  print(Emotion_Voice_Detection_Model.predict([new_feature]))
except:
  print("An exception occurred")

An exception occurred


In [24]:
from scipy.io import wavfile

fs, data = wavfile.read('C:/Users/R.N.Mandal/Desktop/AVER/audio_chunks/audio_ratantata/chunk6.wav')# reading the file

wavfile.write('chunk14_1.wav', fs, data[:, 0])   # saving first column which corresponds to channel 1
wavfile.write('chunk14_2.wav', fs, data[:, 1])

In [25]:
file = 'chunk14_1.wav' #channel set to 1
new_feature= extract_feature(file, mfcc=True, chroma=True, mel=True)
try:
  print(Emotion_Model.predict([new_feature]))
except:
  print("An exception occurred")

['angry']


In [5]:
import glob,pickle
from tqdm import tqdm
from scipy.io import wavfile
a=[]
for file in glob.glob("C:\\Users\\R.N.Mandal\\Desktop\\AVER\\clean_audiochunks\\Marc*\\*.wav"):
    file_name = os.path.basename(file)
    print(file_name)
    new_feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
    print(Emotion_Model.predict([new_feature]))

Marc_Mero_chunk0.wav
['angry']
Marc_Mero_chunk1.wav
['angry']
Marc_Mero_chunk2.wav
['angry']
Marc_Mero_chunk3.wav
['angry']
Marc_Mero_chunk4.wav
['angry']
Marc_Mero_chunk5.wav
['angry']
Marc_Mero_chunk6.wav
['angry']
Marc_Mero_chunk7.wav
['happy']


In [126]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
import numpy as np 
import wave, sys 

In [127]:
x, sr = librosa.load("C:\\Users\\R.N.Mandal\\Desktop\\AVER\\clean_audiochunks\\chunk1.wav")
ipd.Audio(x, rate=sr)

In [35]:
import glob,pickle
from tqdm import tqdm
from scipy.io import wavfile
for file in tqdm(glob.glob(r'C:\Users\R.N.Mandal\Desktop\AVER\audio_chunks\\audio_con*\\*.wav')):
    file_name = os.path.basename(file)
    signal , rate = librosa.load(file, sr=44100)
    mask = envelope(signal,rate, 0.0005)
    wavfile.write(filename= r'C:\Users\R.N.Mandal\Desktop\AVER\clean_audiochunks\\convo\\convo1_'+str(file_name), rate=rate,data=signal[mask])

100%|██████████| 135/135 [00:47<00:00,  2.84it/s]


In [33]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask